In [4]:
from langchain import PromptTemplate
# from fastapi import APIRouter, HTTPException
# from pydantic import BaseModel
import logging
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import httpx

In [7]:

def make_callable(inp):
  return lambda x=None: inp

# Define your custom prompt
template = """
        You are a helpful AI assistant.
        Context:
        {context}

        Dialog history:
        {history}

        Human: {human_input}
        AI:
        """

prompt = PromptTemplate(
    input_variables=["context", "history",  "human_input"],
    template=template
)

In [15]:
async def get_chromadb_responce(text: str, top_k: int):
    url = "http://0.0.0.0:8003/db/find_closest/"
    payload = {
        "text": text,
        "top_k": top_k
    }

    async with httpx.AsyncClient() as client:
        response = await client.post(url, json=payload)

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

In [32]:
async def get_chromfdb_documents(text, top_k) -> str:
    result = await get_chromadb_responce(text, top_k )
    result = '\n\n'.join([f'context_{i}: {document}' for i, document in enumerate(res['result']['documents'][0])])

    return result

In [16]:
res = await get_chromadb_responce('niggers')

In [31]:
'\n\n'.join([f'context_{i}: {document}' for i, document in enumerate(res['result']['documents'][0])])

'context_0: i hate niggers\n\ncontext_1: dogs byte'

In [11]:
async def get_llm_responce(prompt):
    print('prompt')
    print(prompt)
    url = "http://0.0.0.0:8001/llm/responce/"
    params = {"prompt": prompt}

    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params)
            

    if response.status_code == 200:
        return response.json()["ai_response"]
    else:
        response.raise_for_status()

In [12]:
async def chat(human_input, context):
    try:
        memory = ConversationBufferMemory()
        #TODO: llm = get_llm()
        make_callable(memory.load_memory_variables({})['history'])
        
        rag_chain = (
            {"context": make_callable(context), 
                'history': make_callable(memory.load_memory_variables({})['history']),
                "human_input": RunnablePassthrough()}
            | prompt
            | get_llm_responce
            | StrOutputParser()
            )
        
        prompt_filled = prompt.format(**{"context": context, 
                                            'history':memory.load_memory_variables({})['history'], 
                                            "human_input":human_input})
        
        logging.info(f'PROMT: {prompt_filled}')

    
        chain_responce = await rag_chain.ainvoke(human_input)
        print('=====')
        print(chain_responce)
        if not isinstance(chain_responce, str):
            raise ValueError("chain_responce is not a string")
        ai_response = chain_responce.replace(prompt_filled, '')

        memory.save_context({"input": human_input}, {"output": ai_response})

        
        return {"ai_response": ai_response}
    except Exception as e:
        print(e)

In [13]:
human_input = 'hinput'
context = 'context'

ob = await chat(human_input, context)

prompt
text='\n        You are a helpful AI assistant.\n        Context:\n        context\n\n        Dialog history:\n        \n\n        Human: hinput\n        AI:\n        '
=====
you ve done it!


In [14]:
ob

{'ai_response': 'you ve done it!'}

In [44]:
import requests

def get_llm_response(prompt):
    print('Prompt:')
    print(prompt)
    url = "http://0.0.0.0:8001/llm/response/"
    params = {"prompt": prompt}

    try:
        response = requests.get(url, params=params)
        print('Response:')
        print(response)
        print('Response Status Code:', response.status_code)
        print('Response Text:', response.text)

        if response.status_code == 200:
            response_json = response.json()
            print('Response JSON:', response_json)
            return response_json.get("ai_response")
        else:
            response.raise_for_status()
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
prompt = "Your prompt here"
response = get_llm_response(prompt)
print('Final Response:', response)


Prompt:
Your prompt here


KeyboardInterrupt: 